In [12]:
import torch
from torch import nn
import tiktoken

In [2]:
GPT_CONFIG_124M = { 
 "vocab_size": 50257, # 词汇表大小
 "context_length": 1024, # 上下文长度
 "emb_dim": 768, # 嵌入维度
 "n_heads": 12, # 注意力头的数量
 "n_layers": 12, # 层数
 "drop_rate": 0.1, # dropout 率
 "qkv_bias": False # 查询-键-值偏置
}

In [10]:
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
    def forward(self, x):
        return x

In [8]:
class DummyLayerNorm(nn.Module):
    def __init__(self, normlized_shape, eps=1e-5):
        super().__init__()
    def forward(self, x):
        return x

In [16]:
class DummyGPT(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_embed = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_embed = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.dropout = nn.Dropout(cfg["drop_rate"])
        self.block = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias = False
        )
    def forward(self, idx):
        batch_size, seq_len = idx.shape
        token_embed = self.token_embed(idx)
        pos_embed = self.pos_embed(
            torch.arange(seq_len, device=idx.device)
        )
        x = token_embed + pos_embed
        x = self.dropout(x)
        x = self.block(x)
        self.norm(x)
        logits = self.out_head(x)
        return logits

In [14]:
tokenizer = tiktoken.get_encoding("gpt2") 
batch = [] 
txt1 = "Every effort moves you" 
txt2 = "Every day holds a" 
batch.append(torch.tensor(tokenizer.encode(txt1))) 
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0) 
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [17]:
torch.manual_seed(123) 
model = DummyGPT(GPT_CONFIG_124M) 
logits = model(batch) 
print("Output shape:", logits.shape) 
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-1.2034,  0.3201, -0.7130,  ..., -1.5548, -0.2390, -0.4667],
         [-0.1192,  0.4539, -0.4432,  ...,  0.2392,  1.3469,  1.2430],
         [ 0.5307,  1.6720, -0.4695,  ...,  1.1966,  0.0111,  0.5835],
         [ 0.0139,  1.6754, -0.3388,  ...,  1.1586, -0.0435, -1.0400]],

        [[-1.0908,  0.1798, -0.9484,  ..., -1.6047,  0.2439, -0.4530],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.3567,  1.2698, -0.6398,  ..., -0.0162, -0.1296,  0.3717],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)
